<a href="https://colab.research.google.com/github/ideablast/NLPer_chatbot/blob/toram/FAQ_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd

In [11]:
wear_data = pd.read_csv("/content/drive/My Drive/wear.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.
store.head()
print(type(wear_data))

(15826, 20)
(8381,) (7445,)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
            customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))

faqs = []

for i in range(len(store_arr)):
    faqs_tmp =[]
    faqs_tmp.append(str(i+1))
    faqs_tmp.append(customer_arr[i])
    faqs_tmp.append(store_arr[i])

    faqs.append(faqs_tmp)

# print(store_arr[-1])
# print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)
faqs

In [15]:
import jpype
from konlpy.tag import Kkma

In [81]:
kkma = Kkma()
filter_kkma = ['NNG', 'NNP','OL','VA','VV','VXV']

def tokenizer_kkma(doc):
    # 꼬꼬마 형태소 분석기가 자바 기반이어서 파이썬에서 자바함수들을 실행할 수 있는 명령어 (jpype) 를 써줘야한다.
    jpype.attachThreadToJVM()       
    token_doc = ["/".join(word) for word in kkma.pos(doc)]
    return token_doc

def tokenize_kkma_noun_verb(doc):
    jpype.attachThreadToJVM()
    token_doc = ["/".join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
    return token_doc

In [82]:
tokenizer_kkma(faqs[0][1])

['신발/NNG',
 '은/JX',
 '여기/NP',
 '있/VV',
 '는/ETD',
 '것/NNB',
 '이/JKS',
 '다예/NNG',
 '이/VCP',
 '요/EFN',
 '?/SF']

In [83]:
token_faqs = [(tokenizer_kkma(row[1]), row[0]) for row in faqs]
print(token_faqs[:5])
tagged_faqs = [TaggedDocument(d,[c]) for d,c in token_faqs]

[(['신발/NNG', '은/JX', '여기/NP', '있/VV', '는/ETD', '것/NNB', '이/JKS', '다예/NNG', '이/VCP', '요/EFN', '?/SF'], '1'), (['230/NR', '이요/NNG'], '2'), (['네/MDN', '봄/NNG', '이/VCP', '니까/ECD', '편하/VA', '게/ECD', '신/VV', '을/ETD', '수/NNB', '있/VV', '는/ETD', '거/NNG'], '3'), (['굽/NNG', '좀/MAG', '높/VA', '은/ETD', '거/NNB', '없/VA', '나요/EFQ', '?/SF'], '4'), (['언제/MAG', '들어오/VV', '아요/EFN', '?/SF'], '5')]


In [84]:
tagged_faqs[:5]

[TaggedDocument(words=['신발/NNG', '은/JX', '여기/NP', '있/VV', '는/ETD', '것/NNB', '이/JKS', '다예/NNG', '이/VCP', '요/EFN', '?/SF'], tags=['1']),
 TaggedDocument(words=['230/NR', '이요/NNG'], tags=['2']),
 TaggedDocument(words=['네/MDN', '봄/NNG', '이/VCP', '니까/ECD', '편하/VA', '게/ECD', '신/VV', '을/ETD', '수/NNB', '있/VV', '는/ETD', '거/NNG'], tags=['3']),
 TaggedDocument(words=['굽/NNG', '좀/MAG', '높/VA', '은/ETD', '거/NNB', '없/VA', '나요/EFQ', '?/SF'], tags=['4']),
 TaggedDocument(words=['언제/MAG', '들어오/VV', '아요/EFN', '?/SF'], tags=['5'])]

In [87]:
# 모델 만들기
# cpu 몇 개 쓸 건지
import multiprocessing
# 내 컴에 있는 cpu 갯수 cores 에 저장
cores = multiprocessing.cpu_count()
# vector_size : 임베딩할 벡터 차원
# negaive : negative sampling
d2v_faqs = doc2vec.Doc2Vec(
    vector_size = 100,
    alpha = 0.025,
    min_alpha = 0.025,
    hs = 1,
    negative = 0,
    dm = 0,
    dbow_words = 1,
    min_count = 1,
    workers = cores,
    seed = 0
)

# 단어 사전 만들기
d2v_faqs.build_vocab(tagged_faqs)

# 모델 학습
for epoch in range(50):
    print(epoch)
    d2v_faqs.train(tagged_faqs,
                   total_examples = d2v_faqs.corpus_count,
                   epochs = d2v_faqs.epochs
                   )
    d2v_faqs.alpha -=0.0025
    d2v_faqs.min_alpha = d2v_faqs.min_alpha

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [100]:
# 들어온 질문 형태소 분석 및 벡터화 
customer_arr_test = customer_arr
store_arr_test = store_arr

percent_of_correct = 0

for q in range(len(customer_arr)):
    test_string = customer_arr_test[q]
    print(test_string)
    print(store_arr_test[q])
    token_test = tokenizer_kkma(test_string)
    predict_vector = d2v_faqs.infer_vector(token_test)
    # 1에 가까울수록 비슷한 값이고, -1에 가까울수록 상반된 값 (correlation 값)
    topn = 5

    result = d2v_faqs.docvecs.most_similar([predict_vector], topn = topn)

    for i in range(topn):
        if result[i][0] == str(q+1):
            print("**{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
            percent_of_correct+=1
        else:
            print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
        # faqs[faqs[0]==int(result[i][0])][1]
    print()

print("정답 확률 : ",percent_of_correct/len(customer_arr))

 신발은 여기 있는 게 다예요?
네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?
**1위. 0.515442430973053, 1, 네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?
2위. 0.4801880717277527, 179, 일반 티는 밖에 있어요
3위. 0.44584110379219055, 31, 네 다예요
4위. 0.3501318097114563, 6852, 네, 모든 점퍼들이 다 그래요
5위. 0.2828092575073242, 91, 5돈 6돈짜리도 있어요 굵은 거 찾으시면 이런 건 어떠세요?

230이요
편하게 신을 수 있는 거 찾으세요?
1위. 0.44026726484298706, 6470, 예
**2위. 0.4236907362937927, 2, 편하게 신을 수 있는 거 찾으세요?
3위. 0.3867950439453125, 4257, 구입 후 3개월 이내에 가능합니다
4위. 0.36264896392822266, 4501, 가능하지만 그대로 드는 것이 더 예뻐요
5위. 0.36159682273864746, 755, 네 봄 신상품 있어요

네 봄이니까 편하게 신을 수 있는 거
이런 건 어떠세요? 이런 거도 신발 무척 편하거든요
**1위. 0.3851553797721863, 3, 이런 건 어떠세요? 이런 거도 신발 무척 편하거든요
2위. 0.34919363260269165, 6032, 네.
3위. 0.3408898711204529, 6404, 달랑달랑 한 것도 잘 나가고 심플한 거 좋아하시면 이런 것도 좋아요
4위. 0.3292657434940338, 7005, 네, 지금 전품목 다 세일이에요
5위. 0.3147093653678894, 3511, 네 택에 써져있어요

굽 좀 높은 거 없나요?
봄 상품은 아직 어른 제품이 많이 안나왔습니다
**1위. 0.32229527831077576, 4, 봄 상품은 아직 어른 제품이 많이 안나왔습니다
2위. 0.30696675181388855, 38, 여기 있는 거밖에 없어요
3위. 0.300063490

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1위. 0.41219455003738403, 322, 없어요 겨울거는 다 나가서 골고루 없어요
2위. 0.39559438824653625, 539, 세일해서 조금 빼드려요
3위. 0.3916963040828705, 640, 25만 원이요
4위. 0.35024505853652954, 443, 네 봄 신상으로 나온 거라서요
5위. 0.3430475890636444, 920, 이거는 신상품이라 세일은 따로 안들어가고 삼만 구천팔백 원이요

이 티는 원사이즈인가요?
예
1위. 0.3904779553413391, 4269, 어떤 제품인지 말씀해 주시겠어요?
2위. 0.375951886177063, 4557, 네 여학생이면 요즘 뜨는 차콜이 어떠세요
3위. 0.35977163910865784, 6193, 할머님분들은 안에 털 들어있는 신발 어때요?
4위. 0.3483350872993469, 3826, 네 이 구두는 어떠세요?
5위. 0.3453375995159149, 4682, 그럼 이걸로 드릴까요?

이 세트제품에서 티셔츠만 구입할 수 있나요?
티셔츠만 이만 육천 원, 세트로 하면 삼만 오천 원이에요
1위. 0.32080015540122986, 1886, 네 맞아요
2위. 0.3008207678794861, 2899, 네
3위. 0.29651597142219543, 2486, 저희가 가방은 다 수입이라서 가방 세일은 따로 안 해요
4위. 0.28972533345222473, 1875, 네 맞습니다
5위. 0.27632030844688416, 2820, 뒤로 메는 백팩이에요

세일상품 없어요 사장님?
전부 다 세일 가격이에요
1위. 0.3128051161766052, 73, 악세서리 어떤 거 찾으세요?
2위. 0.3052825927734375, 4397, 12000원 입니다
3위. 0.29902493953704834, 4288, 네, 잠시만 기다려주세요.
4위. 0.29714417457580566, 2841